# ✈️Análise e Predição de Seleção de Voos — *FlightRank 2025*

Este notebook aborda um desafio de **sistemas de recomendação com ranking**, aplicado ao contexto de **viagens corporativas**. O objetivo é prever qual voo um usuário (viajante de negócios) tem maior probabilidade de selecionar entre múltiplas opções disponíveis em uma busca.

---

## 🎯Problema e Objetivo

Este é um problema de **aprendizado supervisionado com foco em ranking**, no qual as observações estão agrupadas por sessões de busca (`ranker_id`).

- Cada `ranker_id` representa uma busca real por voos.
- Para cada grupo, há múltiplas opções de voo.
- Exatamente **uma** dessas opções foi selecionada (`selected = 1`).

Nosso objetivo é **treinar um modelo que ordene corretamente as opções** para que o voo escolhido pelo usuário esteja entre os primeiros colocados.

---

## 📏Métrica de Avaliação: HitRate@3

A métrica oficial da competição é o **HitRate@3**, que avalia se o voo realmente escolhido aparece entre os **3 primeiros colocados** no ranking do modelo, por grupo de busca.

A fórmula é:

`HitRate@3 = (1 / |Q|) * ∑ 𝟙(rank_i ≤ 3)`

Onde:

- `|Q|` é o número de sessões de busca avaliadas (com mais de 10 voos).
- `rank_i` é a posição que o modelo atribuiu ao voo correto na sessão `i`.
- `𝟙(rank_i ≤ 3)` vale 1 se o voo estiver entre os top-3, e 0 caso contrário.

> **Importante:** somente sessões com **mais de 10 voos** são consideradas na métrica final.

---

In [1]:
import pandas as pd
import os
import subprocess
import zipfile
import matplotlib.pyplot as plt
import lightgbm as lgb
import numpy as np

from itertools import product
from sklearn.model_selection import GroupShuffleSplit
from itertools import chain
from sklearn.model_selection import GroupKFold

In [2]:
# resetando as configurações 
pd.reset_option('display.max_columns')

# setando as configurações de coluna maxima
pd.set_option('display.max_columns', None)

## ⚙️Download e Extração dos Dados

Antes de iniciarmos a análise, precisamos garantir que os dados da competição estejam disponíveis localmente.

Este passo realiza:

1. **Verificação**: confere se os arquivos já foram baixados.
2. **Download automático** via API do Kaggle (caso necessário).
3. **Extração** dos arquivos `.zip` para a pasta `data/aeroclub/`.

> A API do Kaggle foi configurada com o arquivo `kaggle.json` direto no Windows, não sendo utilizada no diretório do Projeto.



In [3]:
def download_files():
    # Define caminhos
    zip_path = "data/aeroclub-recsys-2025.zip"
    extract_path = "data/aeroclub"
    
    # Cria a pasta base se necessário
    os.makedirs("data", exist_ok=True)

    # Verifica se o arquivo .zip já foi baixado
    if not os.path.exists(zip_path):
        print("🔽 Baixando arquivos da competição...")
        subprocess.run([
            "kaggle", "competitions", "download",
            "-c", "aeroclub-recsys-2025",
            "-p", "data"
        ])
    else:
        print("✅ Arquivo ZIP já existe. Pulando download.")

    # Verifica se os arquivos já foram extraídos
    if not os.path.exists(extract_path) or not os.listdir(extract_path):
        print("📦 Extraindo arquivos...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    else:
        print("✅ Arquivos já extraídos. Pulando extração.")

In [4]:
# Executa
download_files()

✅ Arquivo ZIP já existe. Pulando download.
✅ Arquivos já extraídos. Pulando extração.


## 📚 3. Leitura dos Dados

Nesta etapa, realizamos a **leitura do conjunto de dados de treino**, disponível no arquivo `train.parquet`.

Esse dataset contém as informações completas sobre as **sessões de busca por voos**, incluindo:

- Identificadores do voo e do usuário
- Dados da companhia
- Informações da rota e horários
- Preço total e tarifas
- Regras de cancelamento e remarcação
- Indicação de qual voo foi selecionado (`selected = 1`)

> Este será o **dataset principal** utilizado para construção, treinamento e validação do modelo de recomendação.


In [5]:
train = pd.read_parquet("data/aeroclub/train.parquet")

In [6]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif col_type == 'object':
            num_unique = df[col].nunique()
            num_total = len(df[col])
            if num_unique / num_total < 0.5:
                df[col] = df[col].astype('category')
    
    return df
train = reduce_memory_usage(train)

In [7]:
df_train_raw = train.copy()

## 🧹 4. Seleção de Colunas Relevantes

Com o conjunto de dados carregado, o próximo passo é **selecionar apenas as colunas mais relevantes** para o modelo de baseline.

O foco está em manter variáveis que forneçam **informações úteis para a recomendação do voo**, incluindo:

- Identificadores (`Id`, `ranker_id`, `profileId`, etc.)
- Informações do passageiro e da empresa
- Detalhes da rota, horários e conexões
- Dados de preço, taxas e políticas de reembolso
- Informações do segmento do voo (companhia, assento, bagagem)
- Variável alvo: `selected` (indica o voo escolhido pelo usuário)

> Essa etapa reduz a dimensionalidade e melhora a performance do modelo ao eliminar colunas irrelevantes ou redundantes.

---

### Amostragem opcional para prototipagem

Durante a fase de testes, é comum trabalhar com uma **amostra reduzida dos dados** para agilizar experimentos. Para isso, criamos uma função auxiliar que permite:

- Selecionar apenas as colunas desejadas
- Opcionalmente limitar a quantidade de linhas carregadas

### Exemplos:

```python
# Carregar apenas uma amostra (1 milhão de linhas)
df_train = load_subset(df_train_raw, columns_to_keep, max_rows=1_000_000)

# Carregar o dataset completo (todas as linhas)
df_train = load_subset(df_train_raw, columns_to_keep)


In [8]:
# Define as colunas que você quer manter
columns_to_keep = [
    # Identifiers
    'Id',  # num
    'ranker_id', 
    'profileId', 
    'companyID',
    
    # User info
    'sex', 'nationality', 'frequentFlyer', 'isVip', 'bySelf', 'isAccess3D',

    # Company info
    'corporateTariffCode',

    # Search & route
    'searchRoute', 'requestDate',

    # Pricing
    'totalPrice', 'taxes',

    # Flight timing
    'legs0_departureAt', 'legs0_arrivalAt', 'legs0_duration',
    'legs1_departureAt', 'legs1_arrivalAt', 'legs1_duration',

    # Segment-level info (só do segmento 0 da ida para simplificar no baseline)
    'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_arrivalTo_airport_iata',
    'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_marketingCarrier_code',
    'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_aircraft_code',
    'legs0_segments0_flightNumber',
    'legs0_segments0_duration',
    'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments0_baggageAllowance_weightMeasurementType',
    'legs0_segments0_cabinClass',
    'legs0_segments0_seatsAvailable', 
    'legs0_segments1_departureFrom_airport_iata',
    'legs0_segments2_departureFrom_airport_iata',
    'legs0_segments3_departureFrom_airport_iata',

    # Cancellation & exchange rules
    'miniRules0_monetaryAmount', 'miniRules0_percentage', 'miniRules0_statusInfos',
    'miniRules1_monetaryAmount', 'miniRules1_percentage', 'miniRules1_statusInfos',

    # Pricing policy
    'pricingInfo_isAccessTP', 'pricingInfo_passengerCount',

    # Target
    'selected'
]

# Filtra os dados para o baseline
def load_subset(df, columns,  max_rows=None):
    if max_rows:
        return df[columns].iloc[:max_rows].copy()
    else:
        return df[columns].copy()

# Exemplo de uso
df_train = load_subset(df_train_raw, columns_to_keep, max_rows=1_000_000) # apenas uma parte
#df_train = load_subset(df_train_raw, columns_to_keep) # todos os registros

## 🛠️ 5. Engenharia de Features

Nesta etapa, realizamos a transformação de colunas brutas em variáveis mais informativas, consistentes e apropriadas para uso em modelos de machine learning.

As features serão construídas em subtópicos, organizadas por tipo de transformação.

---

### 🧾 5.1 Correção de Tipos de Dados (`dtypes`)

O primeiro passo é garantir que os tipos de dados estejam corretos e otimizados.

- Colunas categóricas codificadas como `category` são analisadas e convertidas para:
  - `int` ou `float`, quando possível
  - `bool`, se contiverem apenas valores lógicos
  - `category`, em outros casos

- A coluna `nationality`, que chega como número inteiro, é convertida para `string` para preservar sua semântica categórica.

> Essa padronização é essencial para evitar erros e garantir que o modelo interprete corretamente as variáveis.



In [9]:
def fix_column_types(df):
    df_fixed = df.copy()
    for col in df.columns:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            # Tenta converter para tipo numérico
            try:
                df_fixed[col] = pd.to_numeric(df[col])
            except:
                # Se não for numérico, tenta bool
                unique_vals = df[col].dropna().unique()
                if set(unique_vals) <= {True, False}:
                    df_fixed[col] = df[col].astype(bool)
                else:
                    df_fixed[col] = df[col].astype(str)
    return df_fixed
df_train = fix_column_types(df_train)

# Ajusta a nacionalidade (está em Int)
df_train["nationality"] = df_train["nationality"].astype("str")
df_train['companyID'] = df_train['companyID'].astype('category')

df_train.dtypes  # Checar resultado

Id                                                                 int32
ranker_id                                                         object
profileId                                                          int32
companyID                                                       category
sex                                                                 bool
nationality                                                       object
frequentFlyer                                                     object
isVip                                                               bool
bySelf                                                              bool
isAccess3D                                                          bool
corporateTariffCode                                                Int64
searchRoute                                                       object
requestDate                                               datetime64[ns]
totalPrice                                         

### 🛫 5.2 Quantidade de Segmentos na Ida

Neste passo, criamos variáveis relacionadas à **estrutura do voo de ida**, com foco na quantidade de conexões realizadas.

#### O que está sendo feito:

- **`n_segments_ida`**: calcula o número total de segmentos no trecho de ida.
  - Por definição, todo voo possui ao menos o **segmento 0** (origem até primeiro destino).
  - Caso existam conexões (segmentos 1, 2 ou 3), esse número é incrementado.
  
- **`has_connections_ida`**: variável booleana indicando se o voo de ida possui uma ou mais conexões.

- Após a extração das informações, as colunas auxiliares com os segmentos 1 a 3 são removidas, pois não são mais necessárias diretamente.

> Essas features ajudam o modelo a diferenciar voos diretos de voos com escalas, o que pode influenciar a decisão do viajante corporativo.


In [10]:
# 1. Cria a feature 'n_segments_ida' baseada na presença de segmentos 1, 2, 3
segments_ida_cols = {
    1: 'legs0_segments1_departureFrom_airport_iata',
    2: 'legs0_segments2_departureFrom_airport_iata',
    3: 'legs0_segments3_departureFrom_airport_iata'
}

# Começa com 1 porque o segmento 0 está sempre presente
df_train['n_segments_ida'] = 1

for seg, col in segments_ida_cols.items():
    df_train['n_segments_ida'] += df_train[col].notnull().astype(int)

# 2. Cria a flag booleana se há conexões (mais de 1 segmento na ida)
df_train['has_connections_ida'] = (df_train['n_segments_ida'] > 1).astype('boolean')

df_train.drop('legs0_segments1_departureFrom_airport_iata', inplace=True, axis=1)
df_train.drop('legs0_segments2_departureFrom_airport_iata', inplace=True, axis=1)
df_train.drop('legs0_segments3_departureFrom_airport_iata', inplace=True, axis=1)

### 🎫 5.3 Programas de Milhagem (`frequentFlyer`)

A coluna `frequentFlyer` indica os programas de fidelidade aos quais o passageiro está associado. Como ela contém múltiplos códigos concatenados por "/", é necessário processá-la para gerar features mais informativas e utilizáveis pelo modelo.

#### 🧠 Transformações aplicadas:

- **`frequentFlyer_count`**: indica quantos programas de milhagem o passageiro participa (quantidade de códigos separados por "/").
  
- **`hasFrequentFlyer`**: variável binária que indica se o passageiro participa de ao menos um programa.

- **`ff_XXX`**: colunas booleanas individuais para cada companhia aérea, representando se o passageiro é membro do respectivo programa de fidelidade.

- Após extração das features, a coluna original `frequentFlyer` é removida do dataset, já que sua informação foi decomposta em variáveis mais específicas.

> Essas features ajudam a capturar a afinidade do usuário com companhias específicas, o que pode influenciar fortemente sua decisão de voo.


In [11]:
def count_frequent_flyers(value):
    if pd.isna(value):
        return 0
    return len(str(value).split('/'))

df_train['frequentFlyer_count'] = df_train['frequentFlyer'].apply(count_frequent_flyers)

# Cria flag binária para frequent flyer
df_train['hasFrequentFlyer'] = df_train['frequentFlyer'].notnull().astype(int)

# Substituir valores NaN por string vazia
ff_series = df_train['frequentFlyer'].fillna('').astype(str)

# Dividir por '/' para obter lista
ff_lists = ff_series.str.split('/')

all_programs = set(chain.from_iterable(ff_lists))
print(f"Total de companhias únicas: {len(all_programs)}")

df_train.drop('frequentFlyer', axis=1, inplace=True)

Total de companhias únicas: 41


### ⏰ 5.4 Processamento de Datas, Horários e Durações

Nesta etapa, extraímos informações temporais das colunas de data e hora, além de transformar colunas de duração em formatos numéricos.

#### 📅 Transformações aplicadas:

- **Conversão para datetime**: as colunas `requestDate`, `legs0_departureAt`, `legs0_arrivalAt`, `legs1_departureAt` e `legs1_arrivalAt` são convertidas para o tipo `datetime`.

- **Criação de novas variáveis temporais**:
  - `legs0_dep_hour` / `legs1_dep_hour`: hora da partida (ida e volta).
  - `legs0_dep_dayofweek` / `legs1_dep_dayofweek`: dia da semana da partida.
  - `trip_days`: duração da viagem, em dias (volta - ida).
  - `booking_to_trip_days`: número de dias entre a data da busca e a partida.

- **Flags binárias**:
  - `ida_fds` / `volta_fds`: indica se o voo ocorre no final de semana.
  - `ida_comercial` / `volta_comercial`: indica se o voo ocorre durante o horário comercial (entre 07h e 19h).

#### ⏱️ Conversão de durações para minutos

- As colunas `legs0_duration` e `legs1_duration` (formato texto) são convertidas para **duração total em minutos**, tornando-se variáveis numéricas.

- A coluna `legs0_segments0_duration`, correspondente ao primeiro segmento do voo de ida, também é convertida para minutos em uma nova variável: `legs0_duration_minutes`.

> O tratamento de variáveis temporais é essencial para capturar padrões de comportamento, como preferências por voos diurnos, viagens curtas ou agendamento com antecedência.


In [12]:
# 🗓️ Colunas de datas e horários
cols_datetime = [
    'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt',
    'legs1_departureAt', 'legs1_arrivalAt'
]
def process_datetime_and_duration(df):
    df_processed = df.copy()

    # Datas para datetime
    for col in cols_datetime:
        df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce')

    # Features de hora e dia da semana
    df_processed['legs0_dep_hour'] = df_processed['legs0_departureAt'].dt.hour
    df_processed['legs0_dep_dayofweek'] = df_processed['legs0_departureAt'].dt.dayofweek
    df_processed['legs1_dep_hour'] = df_processed['legs1_departureAt'].dt.hour
    df_processed['legs1_dep_dayofweek'] = df_processed['legs1_departureAt'].dt.dayofweek

    # Dias entre ida e volta (duração da viagem)
    df_processed['trip_days'] = (df_processed['legs1_departureAt'] - df_processed['legs0_departureAt']).dt.days

    # Dias de antecedência (request → ida)
    df_processed['booking_to_trip_days'] = (df_processed['legs0_departureAt'] - df_processed['requestDate']).dt.days

    # Final de semana (ida/volta)
    df_processed['ida_fds'] = df_processed['legs0_dep_dayofweek'].isin([5, 6]).astype(int)
    df_processed['volta_fds'] = df_processed['legs1_dep_dayofweek'].isin([5, 6]).astype(int)

    # Horário comercial (7h às 19h)
    def is_business_hour(hour):
        return int(7 <= hour <= 19)

    df_processed['ida_comercial'] = df_processed['legs0_dep_hour'].apply(is_business_hour)
    df_processed['volta_comercial'] = df_processed['legs1_dep_hour'].apply(is_business_hour)

    # ⏱️ Converter colunas de duração para minutos
    def clean_and_convert_duration(col):
        return (
            col
            .fillna("00:00:00")
            .astype(str)
            .str.strip()
            .str.replace("nan", "00:00:00")
            .pipe(pd.to_timedelta, errors='coerce')
            .dt.total_seconds() / 60  # minutos
        )

    cols_duration = ['legs0_duration', 'legs1_duration']
    for col in cols_duration:
        df_processed[col] = clean_and_convert_duration(df_processed[col])

    return df_processed

In [13]:
df_train['legs0_duration_minutes'] = (
    pd.to_timedelta(
        df_train['legs0_segments0_duration'].fillna("00:00:00").astype(str).str.strip(),
        errors='coerce'
    ).dt.total_seconds() / 60  # em minutos
)

df_train.drop('legs0_segments0_duration', axis=1, inplace=True)

In [14]:
# ✅ Applicação
df_train = process_datetime_and_duration(df_train)
df_train.drop(columns=cols_datetime, inplace=True)

### 📍 5.5 Tratamento da Rota de Busca (`searchRoute`)

A coluna `searchRoute` representa a rota completa da viagem pesquisada, tanto de ida quanto de volta, e é codificada como uma string no formato:

IDA/VOLTA → Ex: "GRUFOR/FORGRU" ou "GRUCGHFOR/FORGRUCGH"

    
#### 🛠️ Transformações realizadas:

- A coluna `searchRoute` é convertida para o tipo `string` para garantir consistência.

- A string é dividida em duas partes:
  - **`route_ida`**: trecho de ida da viagem
  - **`route_volta`**: trecho de volta da viagem (se houver)

- De cada trecho, são extraídos:
  - **`ida_from`** e **`ida_to`**: origem e destino da ida
  - **`volta_from`** e **`volta_to`**: origem e destino da volta

- Foi criada também uma variável auxiliar `searchRoute_count`, que indicava o número de trechos presentes na rota (via split por "/"), usada apenas para verificação e removida em seguida.

- A coluna original `searchRoute` foi descartada após a decomposição das informações.

> Essa decomposição permite capturar padrões de origem e destino, bem como comportamentos distintos em rotas com múltiplos trechos — informações valiosas para a tarefa de recomendação.

In [15]:
df_train['searchRoute'] = df_train['searchRoute'].astype(str)
df_train['searchRoute_count'] = df_train['searchRoute'].apply(lambda x: x.split("/"))
df_train['searchRoute_count'] = df_train['searchRoute_count'].apply(lambda x: len(x))
print(f" min {min(df_train['searchRoute_count'])}")
print(f" max {max(df_train['searchRoute_count'])}")
df_train.drop('searchRoute_count', axis=1, inplace=True)

 min 1
 max 2


In [16]:
# Garante que searchRoute está como string
df_train['searchRoute'] = df_train['searchRoute'].astype(str)

# Separa ida e volta
df_train[['route_ida', 'route_volta']] = df_train['searchRoute'].str.split('/', expand=True)

# Extrai origem e destino da ida
df_train['ida_from'] = df_train['route_ida'].str[:3]
df_train['ida_to'] = df_train['route_ida'].str[3:]

# Extrai origem e destino da volta (se existir)
df_train['volta_from'] = df_train['route_volta'].str[:3]
df_train['volta_to'] = df_train['route_volta'].str[3:]

df_train.drop('searchRoute', axis=1, inplace=True)

In [17]:
df_train.head()

,Id,ranker_id,profileId,companyID,sex,nationality,isVip,bySelf,isAccess3D,corporateTariffCode,totalPrice,taxes,legs0_duration,legs1_duration,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_aircraft_code,legs0_segments0_flightNumber,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,selected,n_segments_ida,has_connections_ida,frequentFlyer_count,hasFrequentFlyer,legs0_duration_minutes,legs0_dep_hour,legs0_dep_dayofweek,legs1_dep_hour,legs1_dep_dayofweek,trip_days,booking_to_trip_days,ida_fds,volta_fds,ida_comercial,volta_comercial,route_ida,route_volta,ida_from,ida_to,volta_from,volta_to
0,0,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,16884.0,370.0,160.0,155.0,TLK,KJA,KJA,KV,KV,YK2,216,1.0,0.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,1,3,True,3,1,160.0,15,5,9.0,1.0,23.0,29,1,0,1,1,TLKKJA,KJATLK,TLK,KJA,KJA,TLK
1,1,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,51125.0,2240.0,445.0,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,1.0,1,0,3,True,3,1,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0,TLKKJA,KJATLK,TLK,KJA,KJA,TLK
2,2,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,53695.0,2240.0,445.0,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,1.0,1,0,3,True,3,1,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0,TLKKJA,KJATLK,TLK,KJA,KJA,TLK
3,3,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,81880.0,2240.0,445.0,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1,0,3,True,3,1,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0,TLKKJA,KJATLK,TLK,KJA,KJA,TLK
4,4,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,86070.0,2240.0,445.0,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1,0,3,True,3,1,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0,TLKKJA,KJATLK,TLK,KJA,KJA,TLK


## 🧪 6. Preparação para o Treinamento

Com todas as features já processadas, o próximo passo é preparar os dados para o treinamento do modelo de ranking com LightGBM.

---

### 🎯 6.1 Definição da Variável Alvo e Grupos

- **`target_col`**: variável alvo que indica se o voo foi selecionado (`selected = 1`).
- **`group_col`**: identifica cada sessão de busca de voos (`ranker_id`), usada para agrupar corretamente as opções no modelo de ranking.

---

### 🧮 6.2 Organização das Features

As features são separadas em três tipos:

- 🔢 **Numéricas (`numeric_cols`)**: valores contínuos como preço, duração, número de bagagens, etc.
- 🏷️ **Categóricas (`categorical_cols`)**: variáveis representando códigos, aeroportos, companhias, etc.
- ✅ **Booleanas (`boolean_cols`)**: variáveis indicadoras (ex: `isVip`, `ida_fds`, `hasFrequentFlyer`, etc.)

Essas listas são combinadas na variável final `features`, que será usada como input no modelo.

---

### 🧪 6.3 Separação em treino e validação

Utiliza-se `GroupShuffleSplit` para realizar o **split respeitando os grupos (`ranker_id`)**, garantindo que todas as opções de uma mesma busca fiquem **ou no treino, ou na validação**.

---

### 📦 6.4 Construção dos Datasets para LightGBM

Cria-se os objetos `train_dataset` e `val_dataset`, que são estruturas otimizadas do LightGBM para ranking:

- Incluem os dados (`X_train`, `X_val`) e os targets (`y_train`, `y_val`)
- Recebem a lista de colunas categóricas
- Incorporam os grupos (`group=...`) obrigatórios para tarefas de **ranking supervisionado**
- Definem o parâmetro `max_bin`, que controla a discretização de variáveis contínuas (usado para acelerar o treinamento e permitir uso de GPU)

> Essa estrutura é essencial para utilizar o **objetivo `lambdarank`** corretamente, já que o modelo precisa conhecer os grupos de comparação.



In [18]:
# --- Target e grupo
target_col = "selected"
group_col = "ranker_id"

# --- Categóricas para LightGBM
categorical_cols = [
    'companyID',
    'nationality',
    'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_arrivalTo_airport_iata',
    'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_marketingCarrier_code',
    'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_aircraft_code',
    'corporateTariffCode',
    
    # novas features categóricas da searchRoute
    'route_ida',
    'route_volta',
    'ida_from',
    'ida_to',
    'volta_from',
    'volta_to'
]

# --- Booleanas e numéricas
boolean_cols = [
    'sex', 'isVip', 'bySelf',
    'pricingInfo_isAccessTP', 'hasFrequentFlyer',
    'ida_fds', 'volta_fds',
    'ida_comercial', 'volta_comercial',
    'isAccess3D',
    'has_connections_ida'
] + [col for col in df_train.columns if col.startswith("ff_")]

numeric_cols = [
    'totalPrice', 'taxes',
    'legs0_duration', 'legs1_duration',
    'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments0_baggageAllowance_weightMeasurementType',
    'legs0_segments0_cabinClass',
    'legs0_segments0_seatsAvailable',
    'miniRules0_monetaryAmount', 'miniRules0_percentage',
    'miniRules1_monetaryAmount', 'miniRules1_percentage',
    'booking_to_trip_days', 'trip_days',
    'legs0_dep_hour', 'legs0_dep_dayofweek',
    'legs1_dep_hour', 'legs1_dep_dayofweek',
    'frequentFlyer_count', 'legs0_duration_minutes'
]
features = numeric_cols + categorical_cols + boolean_cols

# --- Converte categóricas para category
for col in categorical_cols:
    df_train[col] = df_train[col].astype("category")

In [19]:
# --- Separação por grupo (ranker_id)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(df_train, groups=df_train["ranker_id"]))

df_train_split = df_train.iloc[train_idx].copy()
df_val = df_train.iloc[val_idx].copy()

# --- Features e targets
X_train = df_train_split[features]
y_train = df_train_split[target_col]
groups_train = df_train_split[group_col].value_counts().sort_index().values

X_val = df_val[features]
y_val = df_val[target_col]
groups_val = df_val[group_col].value_counts().sort_index().values

dataset_params = {
    "max_bin": 63 
}

# --- Criação dos Datasets
train_dataset = lgb.Dataset(
    X_train,
    label=y_train,
    group=groups_train,
    categorical_feature=categorical_cols,
    params=dataset_params  # 💡 AQUI é onde max_bin deve ir também!
)

val_dataset = lgb.Dataset(
    X_val,
    label=y_val,
    group=groups_val,
    categorical_feature=categorical_cols,
    reference=train_dataset,
    params=dataset_params
)



## 🧮 7. Busca de Hiperparâmetros com LightGBM

Antes de treinar o modelo final, realizamos uma **busca em grade (`Grid Search`) manual** para identificar a melhor combinação de hiperparâmetros para o modelo `Lambdarank`.

---

### 🔧 7.1 Parâmetros testados

A busca é realizada sobre os seguintes hiperparâmetros:

- `learning_rate`: taxa de aprendizado (ex: 0.05)
- `num_leaves`: complexidade da árvore (ex: 63, 127)
- `min_data_in_leaf`: regularização baseada no mínimo de amostras por folha (ex: 50, 70, 100)

Todas as combinações possíveis entre esses valores são testadas usando `itertools.product`.

---

### 🧪 7.2 Treinamento e Validação

Para cada combinação de parâmetros:

1. O modelo LightGBM é treinado com:
   - Objetivo: `lambdarank`
   - Métrica: `ndcg@3`
   - Parada antecipada após 50 iterações sem melhoria

2. O desempenho do modelo é avaliado com base no **melhor NDCG@3** obtido na validação.

3. O melhor modelo e conjunto de parâmetros são armazenados.

---

### ✅ 7.3 Resultado da Busca

Ao final da busca:

- É exibida a **melhor combinação de parâmetros**
- É mostrado o **score NDCG@3 obtido**
- É realizada uma **predição de validação** com o modelo vencedor
- Calcula-se a **acurácia top-1**, ou seja, a fração de sessões em que o voo correto ficou em primeiro lugar no ranking do modelo

> Essa avaliação serve como uma verificação prática da qualidade das recomendações antes do treinamento final com todos os dados.


In [20]:
param_grid = {
    'learning_rate': [0.05],
    'num_leaves': [63, 127],              # testa valor maior
    'min_data_in_leaf': [50, 70, 100]     

# Gera todas combinações de parâmetros
param_combinations = list(product(*param_grid.values()))
param_keys = list(param_grid.keys())

best_score = -1
best_model = None
best_params = None

for combo in param_combinations:
    param_set = dict(zip(param_keys, combo))
    print(f"Treinando com: {param_set}")

    params = {
        "objective": "lambdarank",
        "metric": "ndcg",
        "ndcg_eval_at": [3],
        "boosting_type": "gbdt",
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "seed": 42,
        "verbosity": -1,
        "num_threads": 8,
        **param_set
    }

    model = lgb.train(
        params,
        train_dataset,
        valid_sets=[val_dataset],
        valid_names=["valid"],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )

    score = model.best_score["valid"]["ndcg@3"]

    if score > best_score:
        best_score = score
        best_model = model
        best_params = param_set

print("\n✅ Melhor combinação:")
print(best_params)
print(f"NDCG@3: {best_score:.5f}")


Treinando com: {'learning_rate': 0.05, 'num_leaves': 63, 'min_data_in_leaf': 50}
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[281]	valid's ndcg@3: 0.801424
Treinando com: {'learning_rate': 0.05, 'num_leaves': 63, 'min_data_in_leaf': 70}
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[285]	valid's ndcg@3: 0.804463
Treinando com: {'learning_rate': 0.05, 'num_leaves': 63, 'min_data_in_leaf': 100}
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[293]	valid's ndcg@3: 0.806283
Treinando com: {'learning_rate': 0.05, 'num_leaves': 127, 'min_data_in_leaf': 50}
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[250]	valid's ndcg@3: 0.809945
Treinando com: {'learning_rate': 0.05, 'num_leaves': 127, 'min_data_in_leaf': 70}
Training until validation scores don't improve for 50 rounds
Early stopping, best it

In [21]:
# --- Predição
y_pred = model.predict(X_val)

# --- Avaliação Top-1
df_pred = df_val.copy()
df_pred['y_true'] = y_val
df_pred['y_pred'] = y_pred

df_pred_sorted = df_pred.sort_values(['ranker_id', 'y_pred'], ascending=[True, False])
df_top1 = df_pred_sorted.groupby('ranker_id').head(1)

acertos = df_top1['y_true'].sum()
total = df_top1.shape[0]

print(f"Voos escolhidos corretamente (top1): {acertos} de {total} sessões")
print(f"Acurácia top1: {acertos / total:.4f}")

Voos escolhidos corretamente (top1): 590 de 1542 sessões
Acurácia top1: 0.3826


## 🧠 8. Treinamento Final com Validação

Com os melhores hiperparâmetros definidos, realizamos o treinamento completo do modelo `LightGBM` utilizando o objetivo `lambdarank`.

---

### ⚙️ 8.1 Parâmetros definidos para o modelo

O modelo é configurado com:

- `objective = "lambdarank"`: modelo de ranking supervisionado
- `metric = "ndcg"` com `ndcg_eval_at = [3]`: otimiza diretamente a métrica de interesse
- Hiperparâmetros:
  - `"learning_rate": best_params['learning_rate']`',
  - `"num_leaves": best_params['num_leaves']`',
  - `"min_data_in_leaf": best_params['min_data_in_leaf']`,
  - `feature_fraction = 0.8`
  - `bagging_fraction = 0.8`
  - `bagging_freq = 1`
- `early_stopping`: parada antecipada após 120 iterações sem melhoria
- Treinamento até no máximo `num_boost_round = 1000`

---

### 📊 8.2 Avaliação do Modelo

Após o treinamento, é realizada uma predição no conjunto de validação (`X_val`). A avaliação considera:

- Ordenação dos voos dentro de cada grupo (`ranker_id`) com base no score predito (`y_pred`)
- Cálculo da **acurácia top-1**, ou seja, a fração de sessões em que o modelo colocou o voo correto na **primeira posição** do ranking

> Essa métrica serve como uma proxy mais direta para avaliar a eficácia prática do modelo, complementando o NDCG@3.

---

In [22]:
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [3],
    "learning_rate": best_params['learning_rate'],
    "num_leaves": best_params['num_leaves'],
    "min_data_in_leaf": best_params['min_data_in_leaf'],
    "boosting_type": "gbdt",
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": 42,
    "verbosity": -1,
    "num_threads": 8  # ✅ Aqui está correto
}

model = lgb.train(
    params,
    train_dataset,
    valid_sets=[train_dataset, val_dataset],
    valid_names=["train", "valid"],
    num_boost_round=1000, 
    callbacks=[lgb.early_stopping(stopping_rounds=120)],
)


Training until validation scores don't improve for 120 rounds
Early stopping, best iteration is:
[308]	train's ndcg@3: 0.964658	valid's ndcg@3: 0.814894


In [23]:
# --- Predição
y_pred = model.predict(X_val)

# --- Avaliação Top-1
df_pred = df_val.copy()
df_pred['y_true'] = y_val
df_pred['y_pred'] = y_pred

df_pred_sorted = df_pred.sort_values(['ranker_id', 'y_pred'], ascending=[True, False])
df_top1 = df_pred_sorted.groupby('ranker_id').head(1)

acertos = df_top1['y_true'].sum()
total = df_top1.shape[0]

print(f"Voos escolhidos corretamente (top1): {acertos} de {total} sessões")
print(f"Acurácia top1: {acertos / total:.4f}")


Voos escolhidos corretamente (top1): 590 de 1542 sessões
Acurácia top1: 0.3826


## 🏁 9. Treinamento Final com Todo o Dataset

Após validar o modelo e encontrar a melhor configuração de hiperparâmetros, realizamos o **treinamento final utilizando 100% do conjunto de treino**.

---

### 📦 9.1 Dataset Completo

Nesta etapa, utilizamos todas as observações disponíveis:

- `X_full`: todas as features da base `df_train`
- `y_full`: variável alvo (`selected`)
- `groups_full`: estrutura de agrupamento (`ranker_id`) com todos os grupos

Esses dados são convertidos em um `Dataset` do LightGBM para ranking.

---

### 🧠 9.2 Uso do `best_iteration`

Durante a etapa de validação, o modelo treinado com `early_stopping` encontrou um número ideal de iterações — armazenado como `model.best_iteration`.

Esse valor representa o ponto onde o modelo:

- Obteve **melhor performance em validação**
- Antes de começar a **overfitting**

⚠️ Por isso, ao treinar com todo o dataset, usamos:

```python
num_boost_round = model.best_iteration


In [24]:

# ✅ Treinamento final com TODO o dataset de treino
#     usando best_iteration encontrado na validação
# ============================================================

X_full = df_train[features]
y_full = df_train[target_col]
groups_full = df_train[group_col].value_counts().sort_index().values

full_dataset = lgb.Dataset(X_full, y_full, group=groups_full, categorical_feature=categorical_cols)

# ⚠️ Usa o número ideal de iterações do treino anterior
final_model = lgb.train(
    params,
    full_dataset,
    num_boost_round=model.best_iteration 
)

In [25]:
final_model.save_model('modelo_final.txt')

## 📤 10. Geração da Submissão

- Leitura do `modelo_final.txt`.
- Aplicação das mesmas transformações feitas no treino.
- Predição com o modelo final e ordenação por `y_pred`.
- Geração do arquivo `submission.csv` com as posições ranqueadas (`selected`).

In [26]:
model = lgb.Booster(model_file='modelo_final.txt')

In [27]:
# ============================================================
# ## 6. Geração de Submissão
# ============================================================

# 1. Ler test.parquet
df_test = pd.read_parquet("data/aeroclub/test.parquet")

# 2. Aplicar transformações mínimas necessárias
df_test['ranker_id'] = df_test['ranker_id'].astype(str)
df_test['nationality'] = df_test['nationality'].astype(str)
df_test['searchRoute'] = df_test['searchRoute'].astype(str)

# --- Frequent Flyer (mesmos one-hot do treino)
df_test['frequentFlyer'] = df_test['frequentFlyer'].fillna('').astype(str)
ff_lists_test = df_test['frequentFlyer'].str.split('/')

In [ ]:
all_programs = set(chain.from_iterable(ff_lists))
print(f"Total de companhias únicas: {len(all_programs)}")


for program in all_programs:
    if program == '':
        continue
    df_test[f'ff_{program}'] = ff_lists_test.apply(lambda x: int(program in x))

for col in [col for col in df_test.columns if col.startswith("ff_")]:
    df_test[col] = df_test[col].astype(pd.BooleanDtype())

df_test['frequentFlyer_count'] = df_test['frequentFlyer'].apply(count_frequent_flyers)
df_test['hasFrequentFlyer'] = df_test['frequentFlyer'].notnull().astype(int)
df_test.drop(columns=['frequentFlyer'], inplace=True)



Total de companhias únicas: 41


In [ ]:
# --- Datas
cols_datetime = [
    'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt',
    'legs1_departureAt', 'legs1_arrivalAt'
]
for col in cols_datetime:
    df_test[col] = pd.to_datetime(df_test[col], errors='coerce')

df_test['legs0_dep_hour'] = df_test['legs0_departureAt'].dt.hour
df_test['legs0_dep_dayofweek'] = df_test['legs0_departureAt'].dt.dayofweek
df_test['legs1_dep_hour'] = df_test['legs1_departureAt'].dt.hour
df_test['legs1_dep_dayofweek'] = df_test['legs1_departureAt'].dt.dayofweek
df_test['trip_days'] = (df_test['legs1_departureAt'] - df_test['legs0_departureAt']).dt.days
df_test['booking_to_trip_days'] = (df_test['legs0_departureAt'] - df_test['requestDate']).dt.days
df_test['ida_fds'] = df_test['legs0_dep_dayofweek'].isin([5, 6]).astype(int)
df_test['volta_fds'] = df_test['legs1_dep_dayofweek'].isin([5, 6]).astype(int)

df_test['ida_comercial'] = df_test['legs0_dep_hour'].apply(lambda x: int(7 <= x <= 19))
df_test['volta_comercial'] = df_test['legs1_dep_hour'].apply(lambda x: int(7 <= x <= 19))

df_test.drop(columns=cols_datetime, inplace=True)

In [ ]:
######################################
# Cria a feature 'n_segments_ida' baseada na presença de segmentos 1, 2, 3
segments_ida_cols = {
    1: 'legs0_segments1_departureFrom_airport_iata',
    2: 'legs0_segments2_departureFrom_airport_iata',
    3: 'legs0_segments3_departureFrom_airport_iata'
}

# Começa com 1 porque o segmento 0 está sempre presente
df_test['n_segments_ida'] = 1

for seg, col in segments_ida_cols.items():
    df_test['n_segments_ida'] += df_test[col].notnull().astype(int)

# Cria a flag booleana se há conexões (mais de 1 segmento na ida)
df_test['has_connections_ida'] = (df_test['n_segments_ida'] > 1).astype('boolean')

# companyID como categoria
df_test['companyID'] = df_test['companyID'].astype('category')

# isAccess3D como boolean
df_test['isAccess3D'] = df_test['isAccess3D'].astype('boolean')

df_test.drop('legs0_segments1_departureFrom_airport_iata', inplace=True, axis=1)
df_test.drop('legs0_segments2_departureFrom_airport_iata', inplace=True, axis=1)
df_test.drop('legs0_segments3_departureFrom_airport_iata', inplace=True, axis=1)
########################


# --- Duração
def clean_and_convert_duration(col):
    return (
        col
        .fillna("00:00:00")
        .astype(str)
        .str.strip()
        .str.replace("nan", "00:00:00")
        .pipe(pd.to_timedelta, errors='coerce')
        .dt.total_seconds() / 60
    )

df_test['legs0_duration'] = clean_and_convert_duration(df_test['legs0_duration'])
df_test['legs1_duration'] = clean_and_convert_duration(df_test['legs1_duration'])
df_test['legs0_segments0_duration'] = clean_and_convert_duration(df_test['legs0_segments0_duration'])
df_test['legs0_duration_minutes'] = df_test['legs0_duration']
df_test.drop(columns=['legs0_segments0_duration'], inplace=True)

# --- SearchRoute features
df_test[['route_ida', 'route_volta']] = df_test['searchRoute'].str.split('/', expand=True)
df_test['ida_from'] = df_test['route_ida'].str[:3]
df_test['ida_to'] = df_test['route_ida'].str[3:]
df_test['volta_from'] = df_test['route_volta'].str[:3]
df_test['volta_to'] = df_test['route_volta'].str[3:]
df_test.drop('searchRoute', axis=1, inplace=True)


In [ ]:
df_train['companyID'] = df_train['companyID'].astype('category')
# Converte para booleano (caso ainda não esteja)
df_train['isAccess3D'] = df_train['isAccess3D'].astype('boolean')
# Cria flag indicando se há conexões na ida
df_train['has_connections_ida'] = (df_train['n_segments_ida'] > 1).astype('boolean')

# --- Tipagem
for col in categorical_cols:
    df_test[col] = df_test[col].astype("category")

for col in boolean_cols:
    if col in df_test.columns:
        df_test[col] = df_test[col].astype('boolean')

In [ ]:
# Prever com o modelo
X_test = df_test[features]
df_test['y_pred'] = model.predict(X_test)

# 4. Gerar submissão
df_test_sorted = df_test.sort_values(['ranker_id', 'y_pred'], ascending=[True, False])
df_test_sorted['selected'] = df_test_sorted.groupby('ranker_id').cumcount() + 1

submission = df_test_sorted[['Id', 'ranker_id', 'selected']]
submission.to_csv("submission.csv", index=False)
print("✅ Arquivo de submissão salvo como 'submission.csv'")
